In [1]:
#!pip install transformers datasets evaluate sacrebleu transformers[torch] sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.7/119.7 kB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 14.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 11.3 MB/s eta 0:00:00


In [2]:
from datasets import load_dataset
from transformers import MarianMTModel, MarianTokenizer, MarianConfig, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer,  PreTrainedTokenizerFast, AutoTokenizer
#from tokenizers import decoders, models, normalizers, pre_tokenizers, processors, trainers, Tokenizer
import torch
import evaluate
import numpy as np

In [ ]:
device = "cuda:0"

In [35]:
HEADS = 4
ENCODER_LAYERS = 6
DECODER_LAYERS = 6
HIDDEN_LAYERS = 6
VOCAB_SIZE = 63261
EPOCHS = 50
BATCH_SIZE = 128
INIT_LR = 2e-4
BASE_MODEL = "Helsinki-NLP/opus-mt-es-fi"
TRANSFER_LEARNING = True # Keep all other values consistent with the configurations of the model

In [25]:
#tokenizer = Tokenizer(models.WordPiece(unk_token="[UNK]", sep_token="[SEP]", cls_token="[CLS]", pad_token='[PAD]'))
#tokenizer.normalizer = normalizers.Lowercase()
#tokenizer.pre_tokenizer = pre_tokenizers.Sequence([pre_tokenizers.WhitespaceSplit(), pre_tokenizers.Punctuation()])
#tokenizer.decoder = decoders.WordPiece()
#tokenizer.decode_batch = decoders.WordPiece()
#t_trainer = trainers.WordPieceTrainer(special_tokens=["[UNK]", "[SEP]", "[CLS]", "[PAD]"])
checkpoint = "t5-small"
tokenizer = AutoTokenizer.from_pretrained(checkpoint, model_max_length=128)

In [7]:
books = load_dataset("opus_books", "es-fi")

books = books["train"].train_test_split(test_size=0.2)

In [8]:
books['test'][0]

{'id': '2832',
 'translation': {'es': '-Bueno, ya conoce las historias de los campesinos acerca de un sabueso fantasmal.',
  'fi': '"Oh, te kyllä olette kuullut kansan kertomuksia tuosta kummituskoirasta ja sen retkistä?'}}

In [9]:
pre_trained_config = {
  "_num_labels": 3,
  "activation_dropout": 0.0,
  "activation_function": "swish",
  "add_bias_logits": False,
  "add_final_layer_norm": False,
  "architectures": [
    "MarianMTModel"
  ],
  "attention_dropout": 0.0,
  "bad_words_ids": [
    [
      63260
    ]
  ],
  "bos_token_id": 0,
  "classif_dropout": 0.0,
  "classifier_dropout": 0.0,
  "d_model": 512,
  "decoder_attention_heads": 8 if TRANSFER_LEARNING else HEADS,
  "decoder_ffn_dim": 2048,
  "decoder_layerdrop": 0.0,
  "decoder_layers": 6 if TRANSFER_LEARNING else DECODER_LAYERS,
  "decoder_start_token_id": 63260,
  "decoder_vocab_size": 63261,
  "dropout": 0.1,
  "encoder_attention_heads": 8 if TRANSFER_LEARNING else HEADS,
  "encoder_ffn_dim": 2048,
  "encoder_layerdrop": 0.0,
  "encoder_layers": 6 if TRANSFER_LEARNING else ENCODER_LAYERS,
  "eos_token_id": 0,
  "forced_eos_token_id": 0,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2"
  },
  "init_std": 0.02,
  "is_encoder_decoder": True,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2
  },
  "max_length": 512,
  "max_position_embeddings": 512,
  "model_type": "marian",
  "normalize_before": False,
  "normalize_embedding": False,
  "num_beams": 4,
  "num_hidden_layers": 6 if TRANSFER_LEARNING else HIDDEN_LAYERS,
  "pad_token_id": VOCAB_SIZE - 1,
  "scale_embedding": True,
  "share_encoder_decoder_embeddings": True,
  "static_position_embeddings": True,
  "transformers_version": "4.35.2",
  "use_cache": True,
  "vocab_size": VOCAB_SIZE
}

In [10]:
configuration = MarianConfig(**pre_trained_config)

In [11]:
model = MarianMTModel(configuration)

In [12]:
tuned_model = MarianMTModel.from_pretrained("Helsinki-NLP/opus-mt-es-fi")

In [13]:
parts = list(dict(model.named_parameters()).keys())
parts.remove('model.shared.weight')

def transfer (tuned, to_tune, parts):
  target = dict(to_tune.named_parameters())
  source = dict(tuned.named_parameters())

  for part in parts:
    target[part].data.copy_(source[part].data)

if TRANSFER_LEARNING:
  transfer(tuned_model, model, parts)

In [15]:
#with open('./biblia.txt', 'r', encoding='latin-1') as f:
#  biblia = f.read()

#with open('./biblia_utf-8.txt', 'w', encoding='utf-8') as f:
#  f.write(biblia)

#tokenizer.train(['./biblia_utf-8.txt'], t_trainer)
#fast_tokenizer = PreTrainedTokenizerFast(tokenizer_object=tokenizer)

In [26]:
source_lang = "es"
target_lang = "fi"


def preprocess_function(examples):
    inputs = [example[source_lang] for example in examples["translation"]]
    targets = [example[target_lang] for example in examples["translation"]]
    model_inputs = tokenizer(inputs, text_target=targets, max_length=128, truncation=True)
    return model_inputs

In [27]:
tokenized_books = books.map(preprocess_function, batched=True)

Map:   0%|          | 0/2675 [00:00<?, ? examples/s]

Map:   0%|          | 0/669 [00:00<?, ? examples/s]

In [28]:
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

In [29]:
metric = evaluate.load("sacrebleu")

In [30]:
def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [[label.strip()] for label in labels]

    return preds, labels


def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    result = {"bleu": result["score"]}

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    result["gen_len"] = np.mean(prediction_lens)
    result = {k: round(v, 4) for k, v in result.items()}
    return result

In [36]:
training_args = Seq2SeqTrainingArguments(
    output_dir="my_awesome_opus_books_model",
    evaluation_strategy="epoch",
    learning_rate=INIT_LR,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=EPOCHS,
    predict_with_generate=True,
    fp16=True,
    push_to_hub=False,
)

#fast_tokenizer.add_special_tokens({'pad_token': '[PAD]'})

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_books["train"],
    eval_dataset=tokenized_books["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train().to(device)

OutOfMemoryError: ignored

In [24]:
tokenizer('pN iʼ@@')

{'input_ids': [3, 102, 567, 3, 23, 2, 1741, 1741, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1]}